In [1]:
import numpy as np
import pandas as pd

#import pandas_profiling
import ydata_profiling
from ydata_profiling import ProfileReport
 
from collections import Counter

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.lm import Vocabulary

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt

In [3]:
#import nltk
#nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/danil-
[nltk_data]     pass123/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
stop_words = set(stopwords.words('russian'))

# Read

In [5]:
df_train = pd.read_json("dataset/train.json")

In [6]:
df_train.head(3)

,id,text,label,extracted_part
0,809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
1,854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,{'text': ['Поставщик должен предоставить обесп...
2,4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...


In [7]:
def extracted_p_process(df_extracted_part,column):
    res = []
    for items in list(df_extracted_part[column]):
        if len(items)==1:
            res.append(items[0])
        else:
            raise Error("Only one fragment expected")
    return pd.Series(res)

In [8]:
df_extracted_part = pd.DataFrame.from_records(list(df_train['extracted_part']))
for c in df_extracted_part.columns:
    df_extracted_part[c] = extracted_p_process(df_extracted_part,c)

df_extracted_part.head(3)

,text,answer_start,answer_end
0,Размер обеспечения исполнения контракта 6593.2...,1279,1343
1,Поставщик должен предоставить обеспечение испо...,1222,1318
2,Размер обеспечения исполнения контракта 10.00%,1297,1343


In [9]:
df = pd.concat([df_train,df_extracted_part],axis=1)
df = df.drop(['extracted_part'],axis=1)

df.columns = pd.Index(['id', 'text', 'label', 'text_output', 'answer_start', 'answer_end'])
df.head(5)

,id,text,label,text_output,answer_start,answer_end
0,809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 6593.2...,1279,1343
1,854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,Поставщик должен предоставить обеспечение испо...,1222,1318
2,4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1297,1343
3,184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1304,1350
4,211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1302,1348


## TF IDF

In [10]:
tokens = ["<sw>",  #stopword
          "<sot>", #start of text
          "<eot>", #end of text
          "<bos>", #start of sentence
          "<eos>", #start of sentence
          "<num>", #number
          "<>"     #exclude word
          ]

def preprocess_word(word):
    word = word.lower()
    return word

In [11]:
other_word_dict = {"stopwords":dict(),"numbers":dict(),"symbols":dict(),"synonims":dict()}
word_dict = {"all_words":dict(),"output_words":dict(),"words_after_transform":dict()}

def collect(word):
    collect_stopwords(word)
    collect_numbers(word)
    collect_symbols(word)
    collect_words(word)

    
def collect_words(word):
    if word in word_dict["all_words"].keys():
        word_dict["all_words"][word]+=1
    else:
        word_dict["all_words"][word] = 1

def collect_output_words(word):
    if word in word_dict["output_words"].keys():
        word_dict["output_words"][word]+=1
    else:
        word_dict["output_words"][word] = 1
            
def collect_words_after_transform(word):
    if word in word_dict["words_after_transform"].keys():
        word_dict["words_after_transform"][word]+=1
    else:
        word_dict["words_after_transform"][word] = 1
            
    
def collect_stopwords(word):
    if word in stop_words:
        if word in other_word_dict["stopwords"].keys():
            other_word_dict["stopwords"][word]+=1
        else:
            other_word_dict["stopwords"][word] = 1
            
def collect_numbers(word):
    if any(char.isdigit() for char in word):
        if word in other_word_dict["numbers"].keys():
            other_word_dict["numbers"][word]+=1
        else:
            other_word_dict["numbers"][word] = 1

def collect_symbols(word):
    if (not any(char.isalpha() for char in word) and\
        not any(char.isdigit() for char in word)):
        if word in other_word_dict["symbols"].keys():
            other_word_dict["symbols"][word]+=1
        else:
            other_word_dict["symbols"][word] = 1
            
def collect_synonims():
    pass

In [13]:
#text_output_freq = {"words":[],"freq":[]}
num_of_out_words = 0
for text in df.text_output:
    for w in word_tokenize(text):
        w = preprocess_word(w)
        collect_output_words(w)
        num_of_out_words += 1

In [14]:
#text_freq = {"words":[],"freq":[]}
num_of_all_words = 0
for text,s,e in zip(df.text,df.answer_start,df.answer_end):
    texts = [text[:s],text[e:]]
    for t in texts:
        for w in word_tokenize(t):
            w = preprocess_word(w)
            collect_words(w)
            num_of_all_words += 1

In [29]:
df.text[0][df.answer_start[0]:df.answer_end[0]]

'Размер обеспечения исполнения контракта 6593.25 Российский рубль'

In [15]:
num_of_out_words

23186

In [ ]:
#word_dict["all_words"]
#word_dict["output_words"]

In [ ]:
#word_dict['all_words']

In [16]:
word_dict_a = pd.DataFrame.from_dict(word_dict["all_words"],orient='index',columns=['freq']).reset_index()
word_dict_a['freq'] /= num_of_all_words
word_dict_a['doc_num'] = 0

word_dict_out = pd.DataFrame.from_dict(word_dict["output_words"],orient='index',columns=['freq']).reset_index()
word_dict_out['freq'] /= num_of_out_words
word_dict_out['doc_num'] = 0

In [17]:
word_dict_a

,index,freq,doc_num
0,извещение,0.001487,0
1,о,0.007830,0
2,проведении,0.002048,0
3,открытого,0.000204,0
4,конкурса,0.000205,0
...,...,...,...
21249,icm,0.000002,0
21250,site-x,0.000002,0
21251,d1802,0.000002,0
21252,//roseltorg.ru/,0.000002,0


In [18]:
for i,w in enumerate(word_dict_a.index):
    if w in word_dict_out.index:
        word_dict_a.loc[i,'doc_num'] = 2
    else:
        word_dict_a.loc[i,'doc_num'] = 1

for i,w in enumerate(word_dict_out.index):
    if w in word_dict_a.index:
        word_dict_out.loc[i,'doc_num'] = 2
    else:
        word_dict_out.loc[i,'doc_num'] = 1
(word_dict_a['doc_num']==0).sum(),(word_dict_out['doc_num']==0).sum()

(0, 0)

In [26]:
word_dict_out[word_dict_out['doc_num']==1]

,index,freq,doc_num,idf


In [20]:
word_dict_a['idf'] = np.log(2/word_dict_a['doc_num'])
word_dict_out['idf'] = np.log(2/word_dict_out['doc_num'])